In [7]:

import sys
from pathlib import Path
from dataclasses import dataclass, field
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from utils import load_config, load_datasets
import pandas as pd
import numpy as np
import re
import sklearn as skt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, XGBClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from noise_remover import NoiseRemover
from titanic_preprocessor import TitanicPreprocessor
from feature_engineering import FeatureEngineering
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from xgboost.callback import EarlyStopping

load_config()
dataFrameTrain,dataFrameTest = load_datasets()

X=dataFrameTrain.drop(columns=["Survived"])
Y=dataFrameTrain["Survived"]

#print(dataFrameTest)


ids_to_predict=dataFrameTest["PassengerId"]
seed =42

model= XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",       # modela conteos
    tree_method="hist",  
    colsample_bytree = 0.8,
    n_estimators=14,
    learning_rate=0.1,
    subsample=1.0,
    max_depth=2,
    reg_lambda=10,            # L2 ayuda a estabilizar
    reg_alpha=15,
    enable_categorical=True,
    seed=seed
)

#'max_depth': 2, 'learning_rate': 0.009, 'subsample': 0.8, 'colsample_bytree': 1.0 'n_estimators': 5000 , result: 0.02004158700209857, cv: 0.8438350774622557, ov_cof: 0.8237934904601572
#Best score: 0.02004158700209857
#Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.009, 'max_depth': 2, 'reg_alpha': 15, 'reg_lambda': 0, 'subsample': 0.8}

data_preprocessor = TitanicPreprocessor()
features_preprocessor = FeatureEngineering()

pipeline = Pipeline([('pre-processor', data_preprocessor), ('features', features_preprocessor), ('nosie remover', NoiseRemover()),('rfb', model)])
pipeline.fit(X, Y)
#cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#scores = cross_val_score(pipeline, X, Y, scoring="accuracy", cv=cv, n_jobs=-1)
#print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f},, {scores.mean()+scores.std():.4f}")


     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
5              6       3                                   Moran, Mr. James   
6              7       1                            McCarthy, Mr. Timothy J   
7              8       3                     Palsson, Master. Gosta Leonard   
8              9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9             10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10            11       3                    Sandstrom, Miss. Marguerite Rut   
11            12       1                           B

,steps,"[('pre-processor', ...), ('features', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
